## goto
[论文](https://www.cs.utexas.edu/users/pingali/CS378/2008sp/papers/gotoPaper.pdf)

- The ratio between the rate at which floating point operations (flops) can be
performed by the floating point unit(s) and the rate at which floating point
numbers can be streamed from the level-2 (L2) cache to registers is typically
relatively small. This means that matrix A˜ can be streamed from the L2 cache.

 - All libraries that were timed use assembly-coded inner-kernels (including ATLAS). Compiler
options -fomit-frame-pointer -O3 -funroll-all-loops were used.

- A theory that supports an optimality claim regarding the general approach mentioned in this section can be found in [Gunnels et al. 2001]. In particular, that
paper supports the observation that computation should be cast in terms of the
decision tree given in Fig. 4 if data movement between memory layers is to be optimally amortized.

- However,
the bottom line is that under the simplified assumptions A should occupy as much of the cache as possible and should be roughly square2
, while leaving room in the
cache for at least Bj and Cj .

- Throughout the remainder of the paper, we will assume
that matrices are stored in column-major order.  
row major == channel last  
column major == channel first

- This suggests loading matrix A in the cache layer that is farthest from the registers
(can hold the most data) subject to the constraint that Assumptions (a)–(c) are  
(roughly) met.

cache hit / cache miss  
TBL hit / TLB miss
- __The most significant difference between a cache miss and a TLB miss is that a
cache miss does not necessarily stall the CPU. A small number of cache misses can
be tolerated by using algorithmic ___prefetching___ techniques as long as the data can be
read fast enough from the memory where it does exist and arrives at the CPU by
the time it is needed for computation.__ A TLB miss, by contrast, causes the CPU
to stall until the TLB has been updated with the new address. In other words,
prefetching can mask a cache miss but not a TLB miss.

- The fundamental problem now is that A is typically a submatrix
of a larger matrix, and therefore is not contiguous in memory. This in turn means
that addressing it requires many more than the minimal number of TLB entries.
The solution is to pack A in a contiguous work array, $A^~$.

- The reason for the factor two is that when the next blocks of columns Bj+1 and
Cj+1 are first addressed, the TLB entries that address them should replace those
that were used for Bj and Cj . However, upon completion of Cj := AB˜
j + Cj some
TLB entries related to A˜ will be the least recently used, and will likely be replaced
by those that address Bj+1 and Cj+1. The factor two allows entries related to Bj
and Cj to coexist with those for A˜, Bj+1 and Cj+1 and by the time Bj+2 and Cj+2
are first addressed, it will be the entries related to Bj and Cj that will be least
recently used and therefore replaced.

这是说， 如果TLB中只有A, Cj， Bj，当Bj+1，Cj+1到来时，某些A的元素就会被认为是使用时间距离最久（ the least recently used）的数据，从而被Bj+1，Cj+1取代。  
而如果在TLB中保留A, Cj， Bj，Bj+1，Cj+1。那么 the least recently used就会是Cj， Bj（这应该是必然的）， 那么它们就会取代A的元素被替代，从而不会出现TLB miss。  
但是这其中的问题是，TLB的存取策略恒定是LAST USE FIRST OUT吗？真实的存取策略是否比这个假设更为复杂，这个问题需要查更多资料。

- A similar argument can
be made for this case. Since the limiting factor is more typically the amount of
memory that the TLB can address (e.g., the TLB on a current generation Pentium4
can address about 256Kbytes while the L2 cache can hold 2Mbytes), we do not
elaborate on the details.

在L2cache限制速度之前， TLB就已经限制速度了

- The fact that Cj is not contiguous in memory is
not much of a concern, since that data is not reused as part of the computation of
the gepp operation.
这说明我们可以在计算output时就设计store位置，使得计算完成后的output的顺序就是我们想要的顺序， 而不是需要开销很大的重排工作。  
对以上涉及到的操作所消耗的时间进行测量


- Note that a gepdot-based implementation places a block of C in the L2 cache
and reads and writes each of its elements as a few columns and rows of A and B
are streamed from memory. This requires twice the bandwidth between the L2
cache and registers as do the gebp and gepbbased algorithms.  
之前并没有考虑过字宽的影响·

 - In practice, the algorithm
in Fig. 8 can hide the cost of bringing elements of C from and to memory with
computation while it exposes the packing of B as sheer overhead  
test this

- The algorithm
in Fig. 9 can hide the cost of bringing elements of B from memory, but exposes
the cost of unpacking C as sheer overhead.   
?

 - In addition, the set associativity and cache replacement policy further
limit how much of the L1 cache can be occupied by Bj .In practice, kcnr floating
point numbers should occupy less than half of the L1 cache so that elements of
A˜ and Caux do not evict elements of Bj .


#### hp

- In further discussion, we will pretend that one can place data in a specific cache and keep it there for the duration of computations. In fact, caches retain data using some cache replacement policy that evicts data that has not been recently used. By carefully ordering computations, we can encourage data to remain in cache, which is what happens in practice.
cache repalcement policy 是些什么

The difference here is that while one can explicitly load registers, the movement of data into caches is merely encouraged by careful ordering of the computation, since replacement of data in cache is handled by the hardware, which has some cache replacement policy similar to "least recently used" data gets evicted.

In practice the movement of the data can often be overlapped with computation (this is known as prefetching). However, clearly only so much can be overlapped, so it is still important to make the ratio favorable.

what is this https://github.com/ROCmSoftwarePlatform/MIOpen/issues/1092

- The cost of bringing it into the vector registers from some layer in the memory is mostly inconsequential because a lot of computation is performed before it is written back out.

<img src="https://www.cs.utexas.edu/users/flame/laff/pfhp/images/Week3/BLISPicturePack.png" style="zoom:40%" /> 

- For some computations, there is no opportunity to reuse data to that same extent, which means they are inherently bandwidth bound.

- 同样的在level 2 cache里面不动。不停地在level 2 和level 3 的cache之间搬运，从而需要保证本次和下一次的都要在level 2的cache里面。

任务调度策略指计算任务的调度与并行化。在分块 GEMM 算法中，计算负载
经过分块被划分成多个计算任务，在多线程环境下，这些计算任务将被调度到不
同的线程上并行执行。在 GotoBLAS[29] 与 OpenBLAS[30] 的 GEMM 实现中，计
算任务在一个矩阵维度（代码中体现为一个循环）上被并行化，不同线程分配到
的计算负载是相等的。由于 kernel 函数（单个计算任务）的数据规模约等于 L2
cache 的容量，因此这是一种粗粒度的一维并行化策略。随着单个处理器核数不
断增加，参与计算的线程数目随之增加，这种粗粒度的一维并行化在某些情况下
（如矩阵形状狭长）性能不佳。因此，BLIS[49] 采用了一种细粒度的多维并行化方
法。在 BLIS 库的 GEMM 实现中，kernel 函数的数据规模约等于 L1 cache 的容量，
并且支持通过环境变量配置 GEMM 在多个矩阵维度上同时被并行化。
